# MLPC '20 Project Notebook

In [32]:
import numpy as np
import glob

## 1 Load data

In [38]:
metadata = np.loadtxt('train/npy/metadata.txt', delimiter=',', dtype=int, skiprows=1)
feature_names = np.loadtxt('train/npy/feature_names.txt', dtype=str)
features = [np.load(f) for f in glob.glob('train/npy/*.npy')]
labels = [np.load(f) for f in glob.glob('train/npy/*.npz')]

## 2 Analyse the data

### 2.1 How are features correlated with each other or over time?

In [62]:
# TODO

### 2.2 Do any features seem redundant?

In [61]:
# TODO

### 2.3 Which features correlate with the labels?

In [60]:
# TODO

### 2.4 Are there any outliers (in terms of features, or recordings)?

In [63]:
# TODO

### 2.5 Any other interesting observations?

In [65]:
# TODO

### 2.6 Optional: Any correspondence to metadata?

In [66]:
# TODO

### 2.7 Optional: What about multiple annotations?

In [67]:
# TODO

## 3 Train and evaluate classifiers
Perform systematic experiments, using cross-validation, with at least 4 different learning algorithms (suggestion: one from each major group: Support Vector Machines, Neural Networks, Nearest Neighbor Classifiers, Naive Bayes, Decision Trees, Generalized Linear Models, Linear and Quadratic Discriminant Analysis, etc.; perhaps also an “ensemble method” such as Random Forest). Focus on framewise classification accuracy as the evaluation criterion.

For each algorithm: perform a systematic evaluation of different parameter settings, for those parameters that control the algorithm’s overfitting behaviour. Analyse and document how the parameters affect the learned models, and whether overfitting occurs, and to what extent.

In [68]:
# TODO